In [1]:
from src.data.datasets.aleppo import AleppoDataLoader
import pandas as pd

file_path = "../data_downloads/aleppo_processed.csv"
keep_columns = ["p_num", "date", "bgl"]
aleppo = AleppoDataLoader(
    file_path=file_path, keep_columns=keep_columns, num_validation_days=2
)

train_data = aleppo.train_data
test_data = aleppo.validation_data

Schema([('datetime', Datetime(time_unit='ns', time_zone=None)), ('p_num', String), ('start', Datetime(time_unit='ns', time_zone=None)), ('end', Datetime(time_unit='ns', time_zone=None)), ('bg-0:00', Float64), ('day_start_shift', Datetime(time_unit='ms', time_zone=None))])


/Users/vilo/Documents/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/sampling.py:224: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  pdf = pdf.interpolate(method=interpolation_method)
/Users/vilo/Documents/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/sampling.py:224: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  pdf = pdf.interpolate(method=interpolation_method)
/Users/vilo/Documents/nocturnal-hypo-gly-prob-forecast/src/data/preprocessing/sampling.py:224: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  pdf = pdf.interpolate(method=interpolation_method)
/Users/vilo/Documents/nocturnal-hypo-gly-prob-

In [24]:
y_train = train_data.set_index(["p_num", "datetime"])["bg-0:00"].to_frame()
y_test = test_data.set_index(["p_num", "datetime"])["bg-0:00"].to_frame()

In [10]:
y_train

p_num   datetime           
101_51  2015-04-21 16:52:03     7.28
        2015-04-21 16:57:03     7.44
        2015-04-21 17:02:03     7.56
        2015-04-21 17:07:03     7.28
        2015-04-21 17:12:03     7.28
                               ...  
9_9     2015-07-31 06:34:15    12.86
        2015-07-31 06:39:15    12.89
        2015-07-31 06:44:15    12.86
        2015-07-31 06:49:15    12.83
        2015-07-31 06:54:15    12.83
Name: bg-0:00, Length: 8652331, dtype: float64

In [29]:
from sktime.forecasting.chronos import ChronosForecaster

# loss_callback = LossPlottingCallback()

current_time = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")

timestep = 5  # mins
training_samples = len(y_train)
batch_size = 128_000
steps_per_epoch = training_samples // batch_size

TIME_STEP_SIZE = timestep
SAVE_EVERY_EPOCH = 2  # How many epochs to save

# interval = "05mins" if use_5min else "15mins"
interval = "05mins"
dir_path = f"../../src/models/ttm/{interval}/{current_time}"

chronos_forecaster = ChronosForecaster("amazon/chronos-t5-tiny")

print("selected interval: ", interval)
print(f"Training samples: {training_samples}")
print(f"Batch size: {batch_size}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Saving a checkpoint every {steps_per_epoch * SAVE_EVERY_EPOCH} steps")

selected interval:  05mins
Training samples: 8652331
Batch size: 128000
Steps per epoch: 67
Saving a checkpoint every 134 steps


In [30]:
import numpy as np

HOURS_TO_PREDICT = 6
NUM_STEPS = HOURS_TO_PREDICT * 60 // TIME_STEP_SIZE

fh = np.arange(1, NUM_STEPS + 1)

In [32]:
chronos_forecaster.fit(y=y_train[:500_000], fh=fh)

ChronosForecaster(model_path='amazon/chronos-t5-tiny')

In [33]:
chronos_forecaster.save("estimator")

<zipfile.ZipFile filename='estimator.zip' mode='r'>

## Plot!

In [49]:
import plotly.graph_objects as go


def plot_chronos_predictions_plotly(
    forecaster, y_train, y_test, fh, patient_id=None, n_plots=3
):
    """
    Plot Chronos predictions vs actual blood glucose levels using Plotly,
    handling the case where train/test patients differ.

    Parameters:
    - forecaster: Trained Chronos forecaster.
    - y_train: DataFrame with MultiIndex [p_num, datetime], column 'bg-0:00'.
    - y_test: DataFrame with MultiIndex [p_num, datetime], column 'bg-0:00'.
    - fh: Forecasting horizon used with the Chronos model.
    - patient_id: Optional single patient ID to plot.
    - n_plots: Number of test patients to plot if patient_id is not specified.
    """
    test_patients = y_test.index.get_level_values("p_num").unique()

    patients_to_plot = [patient_id] if patient_id else test_patients[:n_plots]

    for p in patients_to_plot:
        if p not in y_test.index.get_level_values("p_num"):
            print(f"Patient {p} not found in test set.")
            continue

        y_test_p = y_test.loc[p]
        y_train_p = (
            y_train.loc[p] if p in y_train.index.get_level_values("p_num") else None
        )

        if y_train_p is None or y_train_p.empty:
            print(f"No training data available for patient {p}. Skipping.")
            continue

        # Predict using Chronos: fit on patient's training history
        try:
            y_pred_p = forecaster.predict(X=None, y=y_train_p["bg-0:00"], fh=fh)
        except Exception as e:
            print(f"Prediction failed for patient {p}: {e}")
            continue

        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=y_train_p.index,
                y=y_train_p["bg-0:00"],
                mode="lines",
                name="Train Actual",
                line=dict(color="blue"),
            )
        )

        fig.add_trace(
            go.Scatter(
                x=y_test_p.index,
                y=y_test_p["bg-0:00"],
                mode="lines+markers",
                name="Test Actual",
                line=dict(color="green"),
            )
        )

        fig.add_trace(
            go.Scatter(
                x=y_pred_p.index,
                y=y_pred_p.values,
                mode="lines+markers",
                name="Forecast",
                line=dict(color="red", dash="dash"),
            )
        )

        fig.update_layout(
            title=f"Patient {p} — Blood Glucose Forecast",
            xaxis_title="Datetime",
            yaxis_title="BG Level (bg-0:00)",
            legend=dict(x=0, y=1),
            hovermode="x unified",
        )

        fig.show()

In [51]:
plot_chronos_predictions_plotly(
    forecaster=chronos_forecaster,
    y_train=y_train,
    y_test=y_test,
    fh=fh,
    n_plots=2,  # or specify patient_id='some_patient_id'
)

/Users/vilo/Documents/nocturnal-hypo-gly-prob-forecast/.venv/lib/python3.11/site-packages/sktime/libs/chronos/chronos.py:494: UserWarning:

We recommend keeping prediction length <= 64. The quality of longer predictions may degrade since the model is not optimized for it. 



No training data available for patient 101_52. Skipping.
